<a href="https://colab.research.google.com/github/Camillabgarcia/Apache_Spark_com_Python/blob/main/Spark_sistema_de_recomendacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>